In [26]:
from selenium import webdriver
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import duckdb
import re
import time
import pandas as pd
from tabulate import tabulate
import re
import os


In [27]:
driver = webdriver.Chrome()
driver.maximize_window() 
url = 'https://agromonitor.vn/group/17/231/giam-sat-gia'
driver.get(url)
wait = WebDriverWait(driver, 10)

In [28]:
try:
    login_icon = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'login-icon')) 
    )
    login_icon.click()
except Exception as e:
    print("Không tìm thấy biểu tượng đăng nhập:", e)
    driver.quit()
    exit()
time.sleep(2)


In [29]:
# Tìm các trường nhập username, password và nút đăng nhập
username_field = driver.find_element(By.NAME, 'username')  
password_field = driver.find_element(By.NAME, 'password')  
login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')  

# Nhập thông tin đăng nhập
username_field.send_keys('account')
password_field.send_keys('password')

# Nhấn nút đăng nhập
login_button.click()
time.sleep(2)


In [30]:
driver.get(url)
time.sleep(5)

In [ ]:
article_links = []
while True:
        articles = driver.find_elements(By.CLASS_NAME, 'title-box')
        for article in articles:
                link = article.find_element(By.TAG_NAME, 'a')
                title_element = link.find_element(By.XPATH, './/div[@class="title"]')
                title = title_element.get_attribute('title')
                if title.startswith("Giám sát giá Phân bón tuần "):
                        link = link.get_attribute('href')   
                        article_links.append(link)

        next_page_button = driver.find_element(By.XPATH,"//body/div[@id='app']/div/div/div/div/div/div/div/button[3]")
        if next_page_button.is_enabled():
                print(next_page_button.is_enabled())
                next_page_button.click()
        else:
                break
        time.sleep(5)
print(article_links)

In [32]:
len(article_links)

120

In [ ]:
article_links

In [34]:
valid_posts_info = []
for link in article_links:
    try:
        driver.get(link)
        time.sleep(2)  
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')
        print(f"Số lượng bảng tìm thấy: {len(tables)}")
        if tables:
            table = tables[0]

            data = []
            header_row = table.find_all('tr')[0]
            headers = [header.text.strip() for header in header_row.find_all('td')]

            for row in table.find_all('tr')[1:]:
                columns = row.find_all('td')
                if columns: 
                    data.append([column.text.strip() for column in columns])

            df = pd.DataFrame(data, columns=headers)
            df.fillna('', inplace=True)

            # Xác định vị trí của "Sài Gòn (giá bán buôn)" và "Tây Nam Bộ (giá đại lý cấp 1 bán ra)"
            start_index = df.index[df.iloc[:, 0] == 'Sài Gòn (giá bán buôn)'].tolist()
            end_index = df.index[df.iloc[:, 0] == 'Tây Nam Bộ (giá đại lý cấp 1 bán ra)'].tolist()

            # Kiểm tra nếu có chỉ mục của "Sài Gòn (giá bán buôn)" và "Tây Nam Bộ (giá đại lý cấp 1 bán ra)"
            if start_index and end_index:
                start_index = start_index[0]
                end_index = end_index[0]
                # Lấy dữ liệu từ start_index đến ngay trước end_index
                df = df.iloc[start_index:end_index]

            valid_posts_info.append(df)
            
            print(tabulate(df, headers='keys', tablefmt='pretty', showindex=False))
        else:
            print(f"Không tìm thấy bảng nào trên {link}")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xử lý liên kết {link}: {e}")

Số lượng bảng tìm thấy: 0
Không tìm thấy bảng nào trên https://agromonitor.vn/post/461211/giam-sat-gia-phan-bon-tuan-19-7-25-7-2024-gia-nhap-khau-chao-gia-dap-ve-viet-nam-co-xu-huong-tang-chung-o-cac-muc-cao-nguon-hang-tam-thoi-chua-duoc-bo-sung-nhung-chao-gia-dap-van-thien-hoa-tuong-phong-xanh-64-tuan-nay-giam-50-250-d-kg-tuy-loai-do-suc-mua-yeu
Số lượng bảng tìm thấy: 1
+-------------------------------------+-----------------------------------------+-----------------------------------------+---------+
|                                     |          TB tuần 5/7-11/7/2024          |          TB tuần 28/6-4/7/2024          | So sánh |
+-------------------------------------+-----------------------------------------+-----------------------------------------+---------+
|       Sài Gòn (giá bán buôn)        |                                         |                                         |         |
|                 Ure                 |                                         |        

In [328]:
selected_columns_list = [df.iloc[:, :3] for df in valid_posts_info]

In [ ]:
melted_dfs = []
for df in valid_posts_info:
    df_selected = df.iloc[:, :3]
    melted_df = df_selected.melt(id_vars=df_selected.columns[0], 
                                 var_name='week', 
                                 value_name='price')
    melted_df.rename(columns={df_selected.columns[0]: 'product_name'}, inplace=True)
    melted_dfs.append(melted_df)

data_unpivoted = pd.concat(melted_dfs, ignore_index=True)
data_unpivoted = data_unpivoted.drop_duplicates()
data_unpivoted = data_unpivoted.groupby(['week', 'product_name'], as_index=False).agg({'price': 'max'})


print(data_unpivoted)

In [ ]:
def extract_first_date(text):
    text = text.strip()
    
    # Tìm kiếm ngày đầu tiên và ngày cuối cùng với định dạng dd/mm-yyyy
    match = re.search(r'(\d{1,2}/\d{1,2})-(\d{1,2}/\d{1,2})/(\d{4})', text)
    if match:
        start_date_str, end_date_str, end_year_str = match.groups()
        start_day, start_month = map(int, start_date_str.split('/'))
        end_day, end_month = map(int, end_date_str.split('/'))
        end_year = int(end_year_str)

        # Điều chỉnh năm
        if start_month == 12 and end_month == 1:
            year = end_year - 1
        else:
            year = end_year

        return f"{start_day}/{start_month}/{year}"

    # Tìm kiếm ngày đầu tiên và ngày cuối cùng với định dạng dd/mm/yyyy-dd/mm/yyyy
    match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})-(\d{1,2}/\d{1,2}/\d{4})', text)
    if match:
        start_date_str, end_date_str = match.groups()
        return start_date_str

    # Xử lý trường hợp có lỗi định dạng như 'TB tuần 30/1/202022-5/1/2023'
    # Tìm kiếm trong chuỗi với định dạng 'TB tuần dd/mm/yyyydd-mm/yyyy'
    match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})\d{2}-(\d{1,2}/\d{1,2}/\d{4})', text)
    if match:
        # Tách riêng các nhóm tìm được từ biểu thức chính quy
        start_date_str, end_date_str = match.groups()

        # Lấy năm của start_date và end_date
        start_year = start_date_str.split('/')[-1]
        end_year = end_date_str.split('/')[-1]

        # Nếu start_date có nhiều hơn 4 chữ số (do lỗi định dạng)
        if len(start_year) > 4:
            # Sửa lại start_date để lấy năm đúng từ end_date
            start_date_str = start_date_str[:start_date_str.rfind('/')] + '/' + end_year

        return start_date_str

    return None
data_unpivoted['reporting_date'] = data_unpivoted['week'].apply(extract_first_date)

print(data_unpivoted)

In [331]:
selected_columns = ['product_name','price','reporting_date','week']
data_to_print = data_unpivoted[selected_columns]


In [ ]:
def process_price_column(df):
    df['price'] = df['price'].str.extract(r'(\d+)')  
    df['price'] = df['price'].fillna(0).astype(float) 
    return df

data_to_print = process_price_column(data_to_print)

print(data_to_print)

In [ ]:
data_to_print.dtypes

In [334]:
df = data_to_print.copy()
df['reporting_date'] =  pd.to_datetime(df['reporting_date'].str[:10] , format='%d/%m/%Y')
df['product_name'] = df['product_name'].astype('string') 
df['price'] = df['price'].astype('float') 
df['week'] = df['week'].astype('string') 



In [ ]:
df

In [ ]:
df_week = df.copy()
df_week[['week1', 'week2']] = df_week['week'].str.split(';', expand=True)
df_filtered = df_week[df_week['week2'].notna()]

print(df_filtered)

In [ ]:
def extract_date(row):
    if 'Ure' in row['product_name'] or 'Kali' in row['product_name']:
        text = row['week1']
    else:
        text = row['week2']

    date_str = extract_first_date(text)
    if date_str:
        return datetime.strptime(date_str, '%d/%m/%Y')
    else:
        return None
    
def extract_first_date(text):
    text = text.strip()
    
    # Tìm kiếm ngày đầu tiên và ngày cuối cùng với định dạng dd/mm-yyyy
    match = re.search(r'(\d{1,2}/\d{1,2})-(\d{1,2}/\d{1,2})/(\d{4})', text)
    if match:
        start_date_str, end_date_str, end_year_str = match.groups()
        start_day, start_month = map(int, start_date_str.split('/'))
        end_day, end_month = map(int, end_date_str.split('/'))
        end_year = int(end_year_str)

        # Điều chỉnh năm
        if start_month == 12 and end_month == 1:
            year = end_year - 1
        else:
            year = end_year

        return f"{start_day}/{start_month}/{year}"

    # Tìm kiếm ngày đầu tiên và ngày cuối cùng với định dạng dd/mm/yyyy-dd/mm/yyyy
    match = re.search(r'(\d{1,2}/\d{1,2}/\d{4})-(\d{1,2}/\d{1,2}/\d{4})', text)
    if match:
        start_date_str, end_date_str = match.groups()
        return start_date_str
    return None

df_filtered['reporting_date'] = df_filtered.apply(extract_date, axis=1)
print(df_filtered)

In [ ]:
selected_columns = ['product_name','price','reporting_date','week']
data_to_print = df_filtered[selected_columns]
data_to_print


In [ ]:
filtered_df = df[~df['week'].str.contains('Ure')]
filtered_df

In [353]:
filtered_df = df[~df['week'].str.contains('Ure')]
df_new = pd.concat([data_to_print,filtered_df],ignore_index=True)

In [ ]:
df_new.dtypes

In [355]:
df_new['product_name_lower'] = df_new['product_name'].str.lower()

df_new = df_new[~df_new['product_name_lower'].isin(['dap', 'npk', 'sa', 'ure', 'kali','sài gòn (giá bán buôn)'])]

df_new_2 = df_new.drop(columns=['product_name_lower'])

In [357]:
def transform(df_new_2: pd.DataFrame) -> pd.DataFrame:
    if df_new_2 is None or len(df_new_2) == 0:
        return None

    # Đưa DataFrame vào DuckDB và thay thế bảng nếu đã tồn tại
    duckdb.sql("DROP TABLE IF EXISTS df_new_2")
    duckdb.sql("CREATE TABLE df_new_2 AS SELECT * FROM df_new_2")

    sql = r"""
     WITH ProductPrices AS (
        SELECT
            week,
            product_name,
            reporting_date,
            price
        FROM df_new_2
    ),
    flag_zero as (
        SELECT
            a.*,
            case when a.price = 0 or a.price is null then 1 else 0 end as flagzero
        FROM ProductPrices a
    ),


    ClosestNonZero AS (
        SELECT
            d.*,
            case when flagzero = 1 then 
                ( select price from flag_zero b where b.product_name = d.product_name and b.reporting_date = (select max(a.reporting_date) from flag_zero a where a.reporting_date < d.reporting_date and a.flagzero = 0 and a.product_name = d.product_name))
            else d.price end as price2
        FROM flag_zero d
    )
    SELECT
        p.week,
        p.product_name,
        case when reporting_date = '2020-01-30' then '2022-12-30' else p.reporting_date end as reporting_date,
        coalesce(p.price2,0) price
    FROM ClosestNonZero p
    ORDER BY p.reporting_date DESC
    """
##do không transform được đoạn bị lỗi do  # Xử lý trường hợp có lỗi định dạng như 'TB tuần 30/1/202022-5/1/2023' nên case when tạm
    df_new_2 = duckdb.query(sql).to_df()

    return df_new_2


result_df = transform(df_new_2)

print(result_df)

                              week                 product_name  \
0            TB tuần 5/7-11/7/2024    DAP Tường Phong, xanh 64%   
1            TB tuần 5/7-11/7/2024  DAP Vân Thiên Hóa, xanh 64%   
2            TB tuần 5/7-11/7/2024             DAP Đình Vũ xanh   
3            TB tuần 5/7-11/7/2024           Kali Belarus miểng   
4            TB tuần 5/7-11/7/2024  Kali Israel miểng (Vinacam)   
...                            ...                          ...   
1700  TB tuần 30/1/202022-5/1/2023          SA Capro Trung Quốc   
1701  TB tuần 30/1/202022-5/1/2023            SA Nhật – K.Cương   
1702  TB tuần 30/1/202022-5/1/2023            SA Trung Quốc bột   
1703  TB tuần 30/1/202022-5/1/2023                   Ure Cà Mau   
1704  TB tuần 30/1/202022-5/1/2023                   Ure Phú Mỹ   

           reporting_date    price  
0     2024-07-05 00:00:00  16300.0  
1     2024-07-05 00:00:00  18550.0  
2     2024-07-05 00:00:00  14550.0  
3     2024-07-05 00:00:00   8600.0  
4     2024

In [359]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define database connection parameters
engine = sa.create_engine(f"mssql+pyodbc://{account}:{password}@{server}/{'database'}?driver=SQL+SERVER", use_setinputsizes=False)

result_df.to_sql('table_name', engine, if_exists='append', index=False)


133

In [ ]:
print(result_df)